In [1]:
# Importing packages 
import os
import numpy as np
import nilearn
import glob
import nibabel as nib
import pandas as pd 
from sklearn.model_selection import cross_val_score
from nilearn.input_data import NiftiMasker 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
import matplotlib
import multiprocessing
from multiprocessing import Pool
import time
from sklearn import preprocessing
from nilearn.image import index_img



np.seterr(divide='ignore', invalid='ignore')
warnings.filterwarnings("ignore")
matplotlib.use('Agg')


/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#image mask
imag_mask='/Users/nikkibytes/Documents/lab/test_imgs/bin_mask.nii.gz'

#our behavioral csv file 
stim = '/Users/nikkibytes/Documents/lab/test_imgs/sub-001.csv'

#our dataset concatenated image 
dataset='/Users/nikkibytes/Documents/lab/test_imgs/sub-001.nii.gz'

#load behavioral data into a pandas df
behavioral = pd.read_csv(stim, sep="\t")
print(behavioral["Label"].unique())


#grab conditional labels and set up milkshake
behavioral["Label"] = behavioral.replace(['HF_LS_receipt', 'LF_LS_receipt', 'LF_HS_receipt', 'HF_HS_receipt'], 'milkshake')


y = behavioral["Label"]
print(y.unique()) # make sure all the milkshake receipts have been replaced with "milkshake"


#restrict data to our target analysis 
condition_mask = behavioral["Label"].isin(['milkshake', "h20_receipt"])
y = y[condition_mask]


#confirm we have the # of condtions needed
print(y.unique())


['rest' 'h20_pic' 'h20_receipt' 'milkshake_pic' 'HF_LS_receipt' 'rinse'
 'LF_LS_receipt' 'LF_HS_receipt' 'HF_HS_receipt']
['rest' 'h20_pic' 'h20_receipt' 'milkshake_pic' 'milkshake' 'rinse']
['h20_receipt' 'milkshake']


In [3]:

# Split data into train and test samples, using the chunks
condition_mask_train = (condition_mask) & (behavioral['sess'] == 0)
condition_mask_test = (condition_mask) & (behavioral['sess']  == 1)


# Apply this sample mask to X (fMRI data) and y (behavioral labels)
# Because the data is in one single large 4D image, we need to use
# index_img to do the split easily
X_train = index_img(dataset, condition_mask_train)
X_test = index_img(dataset, condition_mask_test)
y_train = y[condition_mask_train]
y_test = y[condition_mask_test]

X_test.shape


(63, 74, 49, 0)

In [ ]:
# Compute the mean epi to be used for the background of the plotting
from nilearn.image import mean_img
background_img = mean_img(dataset)

from nilearn.decoding import SpaceNetClassifier
# Fit model on train data and predict on test data
decoder = SpaceNetClassifier(memory_level=2, n_jobs=4,
                             mask=imag_mask, standardize=True,
                             memory="nilearn_cache", penalty='graph-net')
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100.
print("Graph-net classification accuracy : %g%%" % accuracy)

[NiftiMasker.fit] Loading data from None
[NiftiMasker.fit] Resampling mask
[NiftiMasker.transform_single_imgs] Loading data from Nifti2Image(
shape=(63, 74, 49, 2361),
affine=array([[   3.125,    0.   ,    0.   ,  -96.   ],
       [   0.   ,    3.125,    0.   , -132.   ],
       [   0.   ,    0.   ,    4.   ,  -78.   ],
       
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
________________________________________________________________________________
[Memory] Calling nilearn.decoding.space_net.path_scores...
path_scores(<function _graph_net_logistic at 0x117f46d08>, memmap([[ 1.570759, ..., -0.576198],
        ...,
        [-0.664723, ..., -0.576198]]), 
array([-1, ...,  1]), array([[[False, ..., False],
        ...,
        [False, ..., False]],

       ...,

       [[False, ..., False],
        ...,
        [False, ..., False]]]), 
None, [0.5], array([ 295, ..., 2360]), array([  0, ..., 363]), {'max_iter